In [14]:
import cadspy
import numpy as np
import pandas as pd
import datetime as dt

In [15]:
icw = cadspy.DatabaseConnection(system='ICW', user='U193064')

Enter Password:  ··············


<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 1:** Pre-process **all** the tables (df_lounge_eligibility, df_flight_info, df_country, df_acft_typ).

</font>

</ol>
</div>

In [16]:
sel_query = 'select * from LDB_SBOX_OR.HACKATHON_OPS_'
tables = ['LOUNGE_ELIGIBILITY', 'FLIGHT_INFO','COUNTRY_DECODE','AC_TYPE']       

# Creating a loop to run the SQL query for all tables

In [17]:
for table in tables:
    x = icw.queryToDataframe(sel_query+table)
    display(x.head(2))

,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax
0,BA,57,2019-06-18,LHR,JNB,J,J,,EMER,Tier 2,1
1,BA,247,2019-06-29,LHR,GRU,J,J,Gold,EMER,Tier 2,1


,OPG_ALN_CD,OPG_FLT_NO,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,BA,548,2019-06-03 07:30:00,2019-06-03 07:27:00,LHR,5,FCO,FCO,321,V6,LHRFCO,IT,Italy,MEDITERRANEAN,EUROPE EXC UK
1,BA,490,2019-06-12 10:35:00,2019-06-12 10:31:00,LHR,3,GIB,GIB,320,A3,LHRGIB,GI,Gibraltar,MEDITERRANEAN,EUROPE EXC UK


,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,LHRINV,GB,United Kingdom and Northern Ireland,UK,UK
1,LHRSVO,RU,Russia in Europe,EASTERN EUROPE,EUROPE EXC UK


,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,777,K7,WB,0,48,24,203


In [18]:
#Creating a pandas dataframe from the sql loop output
data = {}
for table in tables:
    data[table] = icw.queryToDataframe(sel_query + table)
    x = data[table]

In [19]:
#Stripping blank spaces in columns that are of data type object or date for all dataframes
for tab in tables:
    for col in data[tab].columns:
        if data[tab][col].dtypes == 'object' and 'DT' not in col:
            data[tab][col] = data[tab][col].str.strip()
    

# Lounge Eligibility Dataframe Pre-Process

In [20]:
data['LOUNGE_ELIGIBILITY']['GMT_UPLIFT_DAY_OF_WEEK'] = pd.to_datetime(data['LOUNGE_ELIGIBILITY']['GMT_UPLIFT_DT']).dt.day_name()
data['LOUNGE_ELIGIBILITY'].head()

,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_UPLIFT_DAY_OF_WEEK
0,BA,57,2019-06-18,LHR,JNB,J,J,,EMER,Tier 2,1,Tuesday
1,BA,247,2019-06-29,LHR,GRU,J,J,Gold,EMER,Tier 2,1,Saturday
2,BA,766,2019-06-13,LHR,OSL,C,C,Silver,SAPP,Tier 3,7,Thursday
3,BA,556,2019-05-21,LHR,FCO,M,M,,SAPP,Tier 3,1,Tuesday
4,BA,27,2019-09-19,LHR,HKG,F,F,Premier,EMER,Tier 1,2,Thursday


# Flight Info Data Frame Pre-Process

In [21]:
#Investigate split of planned departure timestamps
pd.to_datetime(data['FLIGHT_INFO'].GMT_PLND_DEP_TS).dt.hour.value_counts().sort_index()

5.0     1926
6.0     6981
7.0     5033
8.0     3695
9.0     4069
10.0    4226
11.0    3916
12.0    4787
13.0    5132
14.0    5493
15.0    5348
16.0    4017
17.0    3962
18.0    4139
19.0    3909
20.0    2695
21.0    1265
Name: GMT_PLND_DEP_TS, dtype: int64

In [22]:
#Get departure hour
data['FLIGHT_INFO']['GMT_PLND_DEP_HOUR'] = pd.to_datetime(data['FLIGHT_INFO'].GMT_PLND_DEP_TS).dt.hour

In [23]:
#Just renaming the data frames for convenience
df_flight_info = data['FLIGHT_INFO']
df_country = data['COUNTRY_DECODE']
df_acft_typ = data['AC_TYPE']
df_lounge_eligibility = data['LOUNGE_ELIGIBILITY']

In [24]:
#Creating buckets for time of day
def hour_group(row):
    if row['GMT_PLND_DEP_HOUR'] >= 5 and row['GMT_PLND_DEP_HOUR'] <= 8:
        return '1. Early Morning'
    elif row['GMT_PLND_DEP_HOUR'] >= 9 and row['GMT_PLND_DEP_HOUR'] <= 12:
        return '2. Morning'
    elif row['GMT_PLND_DEP_HOUR'] >= 13 and row['GMT_PLND_DEP_HOUR'] <= 16:
        return '3. Afternoon'
    elif row['GMT_PLND_DEP_HOUR'] >= 17 and row['GMT_PLND_DEP_HOUR'] <= 21:
        return '4. Evening'

df_flight_info['GMT_PLND_DEP_HOUR'] = df_flight_info.apply(hour_group,axis=1)
df_flight_info['GMT_UPLIFT_DT'] = pd.to_datetime(df_flight_info['GMT_PLND_DEP_TS']).dt.date

In [25]:
#Group up at haul level
def haul_group(row):
    if row['CORP_GEOG_CONTINENT_NM'] == 'UK':
        return 'DOM'
    elif row['CORP_GEOG_CONTINENT_NM'] == 'EUROPE EXC UK':
        return 'SH'
    else:
        return 'LH'

df_flight_info['HAUL'] = df_flight_info.apply(haul_group,axis=1)

In [26]:
#Remove dupes at Date, Flight level

df_flight_info[df_flight_info.duplicated(subset=['GMT_UPLIFT_DT','OPG_FLT_NO'],keep = False)].sort_values(['GMT_UPLIFT_DT','OPG_FLT_NO']).head(10)

,OPG_ALN_CD,OPG_FLT_NO,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,GMT_PLND_DEP_HOUR,GMT_UPLIFT_DT,HAUL
992,BA,223,2019-04-29 13:45:00,2019-04-29 13:44:00,LHR,5,BNA,LHR,789,L8,LHRBNA,US,United States,USA,NORTH AMERICA,3. Afternoon,2019-04-29,LH
16157,BA,223,2019-04-29 13:45:00,2019-04-29 16:37:00,LHR,5,BNA,BNA,789,L8,LHRBNA,US,United States,USA,NORTH AMERICA,3. Afternoon,2019-04-29,LH
1262,BA,293,2019-05-31 16:10:00,2019-05-31 16:45:00,LHR,5,IAD,LHR,744,V4,LHRIAD,US,United States,USA,NORTH AMERICA,3. Afternoon,2019-05-31,LH
28580,BA,293,2019-05-31 16:10:00,2019-05-31 21:37:00,LHR,5,IAD,IAD,744,V4,LHRIAD,US,United States,USA,NORTH AMERICA,3. Afternoon,2019-05-31,LH
857,BA,874,2019-06-25 11:25:00,2019-06-25 11:29:00,LHR,3,BUD,LHR,320,A3,LHRBUD,HU,Hungary,EASTERN EUROPE,EUROPE EXC UK,2. Morning,2019-06-25,SH
9060,BA,874,2019-06-25 11:25:00,2019-06-25 15:06:00,LHR,3,BUD,BUD,320,A3,LHRBUD,HU,Hungary,EASTERN EUROPE,EUROPE EXC UK,2. Morning,2019-06-25,SH
1127,BA,119,2019-07-03 13:15:00,2019-07-03 14:01:00,LHR,5,BLR,LHR,777,L7,LHRBLR,IN,India,SOUTH ASIA,ASIA SOUTH EAST AND CENTRAL,3. Afternoon,2019-07-03,LH
2955,BA,119,2019-07-03 13:15:00,2019-07-03 20:28:00,LHR,5,BLR,BLR,777,L7,LHRBLR,IN,India,SOUTH ASIA,ASIA SOUTH EAST AND CENTRAL,3. Afternoon,2019-07-03,LH
30,BA,251,2019-07-05 21:00:00,2019-07-05 21:01:00,LHR,5,SCL,LHR,789,L8,LHRSCL,CL,Chile,SOUTH AMERICA,SOUTH AMERICA INC CARIBBEAN,4. Evening,2019-07-05,LH
12703,BA,251,2019-07-05 21:00:00,2019-07-06 16:28:00,LHR,5,SCL,SCL,789,L8,LHRSCL,CL,Chile,SOUTH AMERICA,SOUTH AMERICA INC CARIBBEAN,4. Evening,2019-07-05,LH


In [27]:
#Keep last appearance of flight on day
df_flight_info.drop_duplicates(subset = ['GMT_UPLIFT_DT','OPG_FLT_NO'], keep='last', inplace = True)

# Processing of AC Type Table:

In [28]:
#Group according to AC Type and whether has first
def AC_Type_Group(row):
    if row['WB_NB_CAT'] == 'NB':
        return 'NB'
    elif row['WB_NB_CAT'] == 'WB' and row['FIRST_SEATS_QTY'] == 0:
        return 'WB - No First'
    elif row['WB_NB_CAT'] == 'WB' and row['FIRST_SEATS_QTY'] > 0:
        return 'WB - First'

df_acft_typ['AC_TYP_GRP'] = df_acft_typ.apply(AC_Type_Group,axis=1)

In [29]:
#Check for duplicates
df_acft_typ[df_acft_typ.duplicated(subset=['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],keep = False)].sort_values(['IATA_AC_TYP_CD','ACT_AC_TYP_CD']).head(10)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,AC_TYP_GRP


<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 2:** Join the tables below
   
    - df_flight_info
    - df_country
    - df_acft_typ
    
to the table df_lounge_eligibility to generate a final dataset.

</font>

</ol>
</div>

In [30]:
# joining df_lounge_eligibility and df_flight_info

df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['GMT_UPLIFT_DT','OPERATING_FLT_NO'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['GMT_UPLIFT_DT', 'OPG_FLT_NO'], # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

In [31]:
# joining df_lounge_elig_flight_info with df_country

df_lounge_elig_flight_country_info = pd.merge(df_lounge_elig_flight_info,# left table
                                     df_country, # right table
                                     left_on = 'ROUTE', # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = 'ROUTE', # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

In [32]:
#Join existing table with df_acft_typ
df_lounge_elig_flight_info_country_acft_typ = pd.merge(df_lounge_elig_flight_country_info,# left table
                                     df_acft_typ, # right table
                                     left_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'], # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     ) 

In [33]:
df_lounge_elig_flight_info_country_acft_typ.head()

,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,...,COUNTRY_CD_y,COUNTRY_NM_y,CORP_GEOG_CTRY_GRP_NM_y,CORP_GEOG_CONTINENT_NM_y,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,AC_TYP_GRP
0,BA,57,2019-06-18,LHR,JNB,J,J,,EMER,Tier 2,...,ZA,South Africa,SOUTHERN AFRICA,AFRICA,WB,14.0,86.0,30.0,145.0,WB - First
1,BA,247,2019-06-29,LHR,GRU,J,J,Gold,EMER,Tier 2,...,BR,Brazil,SOUTH AMERICA,SOUTH AMERICA INC CARIBBEAN,WB,14.0,56.0,44.0,183.0,WB - First
2,BA,766,2019-06-13,LHR,OSL,C,C,Silver,SAPP,Tier 3,...,NO,Norway,SCANDINAVIA,EUROPE EXC UK,NB,0.0,38.0,0.0,160.0,NB
3,BA,556,2019-05-21,LHR,FCO,M,M,,SAPP,Tier 3,...,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,38.0,0.0,160.0,NB
4,BA,27,2019-09-19,LHR,HKG,F,F,Premier,EMER,Tier 1,...,HK,Hong Kong - SAR of China,FAR EAST HUBS,ASIA FAR EAST,WB,14.0,56.0,44.0,183.0,WB - First


# Removing Unecessary Columns and Ordering

In [34]:
#These are columns at a level that we don't want to use
df_lounge_elig_flight_info_country_acft_typ.drop(['OPERATING_AIRLINE_CD','OPERATING_FLT_NO','UPLIFT_STN_CD','BOOKED_CABIN_CD','TRAVEL_CABIN_CD',
                                                 'BA_PAX_TIER','ONEWORLD_TIER','OPG_ALN_CD','OPG_FLT_NO','GMT_PLND_DEP_TS','GMT_ACT_DEP_TS',
                                                 'ACT_DEP_STN_CD','PLND_ARR_STN_CD','ACT_ARR_STN_CD','IATA_AC_TYP_CD','ACT_AC_TYP_CD','ROUTE',
                                                 'COUNTRY_CD_x','COUNTRY_CD_y','COUNTRY_NM_y','CORP_GEOG_CTRY_GRP_NM_y','CORP_GEOG_CONTINENT_NM_y',
                                                'WB_NB_CAT','FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY',
                                                 'ECONOMY_SEATS_QTY'],axis=1, inplace = True)

In [35]:
#Rename duplicated columns
df_lounge_elig_flight_info_country_acft_typ.columns = df_lounge_elig_flight_info_country_acft_typ.columns.to_series().replace({'COUNTRY_NM_x':'COUNTRY_NM', 'CORP_GEOG_CTRY_GRP_NM_x':'CORP_GEOG_CTRY_GRP_NM',
                                                             'CORP_GEOG_CONTINENT_NM_x':'CORP_GEOG_CONTINENT_NM'})
#Order day of week to summarise easier
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
from pandas.api.types import CategoricalDtype
df_lounge_elig_flight_info_country_acft_typ['GMT_UPLIFT_DAY_OF_WEEK'] = df_lounge_elig_flight_info_country_acft_typ['GMT_UPLIFT_DAY_OF_WEEK'].astype(CategoricalDtype(categories=cats, ordered=True))


<br><br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">Exercise 3: Based on Summer 2019 data, provide a lookup table of Lounge eligibility assumptions that can be applied to a future schedule. To do so, answer each of the following questions in the Markdown cell provided below. 

- What level of granularity do you use?
- What metric do you use to come up with Lounge eligibility profiles?

    
Note 1: **Provide evidence for your assumptions.** This can be in the form of tables, graphs, correlation matrix, etc.
    
Note 2: Make use of the examples below to give structure to your answer. Feel free to attend the Hackathon Clinics if you have any questions. 
</font>


    
</ol>
</div>

In [36]:
df_lounge_elig_flight_info_country_acft_typ.head()

,GMT_UPLIFT_DT,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax,GMT_UPLIFT_DAY_OF_WEEK,ACT_DEP_TML_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,GMT_PLND_DEP_HOUR,HAUL,AC_TYP_GRP
0,2019-06-18,JNB,Tier 2,1,Tuesday,5,South Africa,SOUTHERN AFRICA,AFRICA,4. Evening,LH,WB - First
1,2019-06-29,GRU,Tier 2,1,Saturday,5,Brazil,SOUTH AMERICA,SOUTH AMERICA INC CARIBBEAN,4. Evening,LH,WB - First
2,2019-06-13,OSL,Tier 3,7,Thursday,5,Norway,SCANDINAVIA,EUROPE EXC UK,2. Morning,SH,NB
3,2019-05-21,FCO,Tier 3,1,Tuesday,5,Italy,MEDITERRANEAN,EUROPE EXC UK,3. Afternoon,SH,NB
4,2019-09-19,HKG,Tier 1,2,Thursday,5,Hong Kong - SAR of China,FAR EAST HUBS,ASIA FAR EAST,4. Evening,LH,WB - First


Demonstrate why want to split by day of week - the % of customers in each tier varies by Day of Week, particularly around the weekend

In [37]:
# Demonstrate why want to split by day of week
list_groupby = ['GMT_UPLIFT_DAY_OF_WEEK','Lounge_eligibility_tier']

# grouping by GMT_UPLIFT_DAY_OF_WEEK and Lounge_eligibility_tier
df_groupby_DOW = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()


# columns that you want to group by
list_groupby = ['GMT_UPLIFT_DAY_OF_WEEK']

# grouping by WB_NB_CAT
df_groupby_DOW_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_DOW = pd.merge(df_groupby_DOW,
                            df_groupby_DOW_ttl,
                            on = ['GMT_UPLIFT_DAY_OF_WEEK'],
                            how = 'left'
                           )

df_groupby_DOW['pax_eligible%'] = (df_groupby_DOW['pax_count'] / df_groupby_DOW['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_DOW['pax_eligible%'] = df_groupby_DOW['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_DOW.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

# using set_index to come up with the final lookup table
df_groupby_DOW = df_groupby_DOW.set_index(['GMT_UPLIFT_DAY_OF_WEEK','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

print(df_groupby_DOW)
print()
print(df_groupby_DOW_ttl)

                        pax_eligible%                     
Lounge_eligibility_tier  Not eligible Tier 1 Tier 2 Tier 3
GMT_UPLIFT_DAY_OF_WEEK                                    
Monday                          70.2%   2.2%   7.5%  20.2%
Tuesday                         70.7%   2.2%   6.6%  20.4%
Wednesday                       70.8%   2.2%   6.4%  20.7%
Thursday                        70.3%   2.1%   6.5%  21.1%
Friday                          70.5%   2.1%   6.7%  20.8%
Saturday                        72.4%   2.0%   5.4%  20.2%
Sunday                          72.3%   2.0%   6.1%  19.5%

  GMT_UPLIFT_DAY_OF_WEEK  pax_count_ttl
0                 Monday        1731299
1                Tuesday        1628300
2              Wednesday        1720472
3               Thursday        1812105
4                 Friday        1825035
5               Saturday        1703945
6                 Sunday        1764454


Demonstrate why want to split by time of day - the % of customers in each tier differs by time of day, particularly early morning

In [38]:
list_groupby = ['GMT_PLND_DEP_HOUR','Lounge_eligibility_tier']

# grouping by GMT_UPLIFT_DAY_OF_WEEK and Lounge_eligibility_tier
df_groupby_TOD = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()


# columns that you want to group by
list_groupby = ['GMT_PLND_DEP_HOUR']

# grouping by WB_NB_CAT
df_groupby_TOD_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_TOD = pd.merge(df_groupby_TOD,
                            df_groupby_TOD_ttl,
                            on = ['GMT_PLND_DEP_HOUR'],
                            how = 'left'
                           )

df_groupby_TOD['pax_eligible%'] = (df_groupby_TOD['pax_count'] / df_groupby_TOD['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_TOD['pax_eligible%'] = df_groupby_TOD['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_TOD.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

# using set_index to come up with the final lookup table
df_groupby_TOD = df_groupby_TOD.set_index(['GMT_PLND_DEP_HOUR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

print(df_groupby_TOD)
print()
print(df_groupby_TOD_ttl)

                        pax_eligible%                     
Lounge_eligibility_tier  Not eligible Tier 1 Tier 2 Tier 3
GMT_PLND_DEP_HOUR                                         
1. Early Morning                69.7%   1.4%   7.9%  21.0%
2. Morning                      69.9%   2.2%   6.5%  21.4%
3. Afternoon                    72.0%   2.2%   5.6%  20.1%
4. Evening                      72.1%   2.6%   6.1%  19.2%

  GMT_PLND_DEP_HOUR  pax_count_ttl
0  1. Early Morning        2654237
1        2. Morning        3072698
2      3. Afternoon        3672724
3        4. Evening        2785902


Demonstrate why want to split by Country Group Name - the % of customers in each tier varies significantly by country group

In [39]:
list_groupby = ['CORP_GEOG_CTRY_GRP_NM','Lounge_eligibility_tier']

# grouping by GMT_UPLIFT_DAY_OF_WEEK and Lounge_eligibility_tier
df_groupby_ctry = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()


# columns that you want to group by
list_groupby = ['CORP_GEOG_CTRY_GRP_NM']

# grouping by WB_NB_CAT
df_groupby_ctry_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_ctry = pd.merge(df_groupby_ctry,
                            df_groupby_ctry_ttl,
                            on = ['CORP_GEOG_CTRY_GRP_NM'],
                            how = 'left'
                           )

df_groupby_ctry['pax_eligible%'] = (df_groupby_ctry['pax_count'] / df_groupby_ctry['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_ctry['pax_eligible%'] = df_groupby_ctry['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_ctry.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

# using set_index to come up with the final lookup table
df_groupby_ctry = df_groupby_ctry.set_index(['CORP_GEOG_CTRY_GRP_NM','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')


print(df_groupby_ctry)
print()
#Check that each country group has enough volume
print(df_groupby_ctry_ttl)

                        pax_eligible%                     
Lounge_eligibility_tier  Not eligible Tier 1 Tier 2 Tier 3
CORP_GEOG_CTRY_GRP_NM                                     
BENELUX                         70.4%   0.9%   8.9%  19.8%
CANADA                          73.7%   2.8%   4.0%  19.4%
CARIBBEAN                       71.9%   0.5%   5.8%  21.8%
CENTRAL AMERICA                 70.0%   5.1%   4.6%  20.3%
CHINA                           69.3%   5.9%   4.7%  20.2%
EAST AND CENTRAL AFRICA         72.4%   4.3%   4.4%  18.9%
EASTERN EUROPE                  77.3%   0.6%   5.4%  16.8%
FAR EAST HUBS                   68.5%   4.9%   5.9%  20.7%
FAR EAST JSA HUBS               69.4%   3.6%   6.5%  20.5%
FAR EAST OTHER                  77.9%   0.4%   4.3%  17.3%
IRELAND RO                      72.7%   0.7%   7.4%  19.3%
JAPAN                           69.8%   4.9%   5.3%  20.0%
MEDITERRANEAN                   71.2%   0.9%   6.6%  21.3%
MIDDLE EAST OTHER               78.2%   0.4%   5.4%  16.

Demonstrate why want to split by AC Type Group - % of customers in each tier varies a lot by aircraft type, particularly whether the aircraft has First



In [40]:
list_groupby = ['AC_TYP_GRP','Lounge_eligibility_tier']

# grouping by GMT_UPLIFT_DAY_OF_WEEK and Lounge_eligibility_tier
df_groupby_ac = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()


# columns that you want to group by
list_groupby = ['AC_TYP_GRP']

# grouping by WB_NB_CAT
df_groupby_ac_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_ac = pd.merge(df_groupby_ac,
                            df_groupby_ac_ttl,
                            on = ['AC_TYP_GRP'],
                            how = 'left'
                           )

df_groupby_ac['pax_eligible%'] = (df_groupby_ac['pax_count'] / df_groupby_ac['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_ac['pax_eligible%'] = df_groupby_ac['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_ac.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

# using set_index to come up with the final lookup table
df_groupby_ac = df_groupby_ac.set_index(['AC_TYP_GRP','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')


print(df_groupby_ac)
print()
print(df_groupby_ac_ttl)

                        pax_eligible%                     
Lounge_eligibility_tier  Not eligible Tier 1 Tier 2 Tier 3
AC_TYP_GRP                                                
NB                              72.8%   0.6%   7.0%  19.6%
WB - First                      67.0%   5.1%   5.8%  22.1%
WB - No First                   78.0%   0.3%   4.2%  17.5%

      AC_TYP_GRP  pax_count_ttl
0             NB        7234921
1     WB - First        4225416
2  WB - No First         595764


Check that volumes are reasonable at overall level



In [41]:
list_groupby = ['GMT_UPLIFT_DAY_OF_WEEK','GMT_PLND_DEP_HOUR','CORP_GEOG_CTRY_GRP_NM','AC_TYP_GRP','Lounge_eligibility_tier']

# grouping by GMT_UPLIFT_DAY_OF_WEEK and Lounge_eligibility_tier
df_groupby_all = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()


# columns that you want to group by
list_groupby = ['GMT_UPLIFT_DAY_OF_WEEK','GMT_PLND_DEP_HOUR','CORP_GEOG_CTRY_GRP_NM','AC_TYP_GRP']

# grouping by WB_NB_CAT
df_groupby_all_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_all = pd.merge(df_groupby_all,
                            df_groupby_all_ttl,
                            on =  ['GMT_UPLIFT_DAY_OF_WEEK','GMT_PLND_DEP_HOUR','CORP_GEOG_CTRY_GRP_NM','AC_TYP_GRP'],
                            how = 'left'
                           )

df_groupby_all['pax_eligible%'] = (df_groupby_all['pax_count'] / df_groupby_all['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_all['pax_eligible%'] = df_groupby_all['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_all.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

# using set_index to come up with the final lookup table
df_groupby_all = df_groupby_all.set_index(['GMT_UPLIFT_DAY_OF_WEEK','GMT_PLND_DEP_HOUR','CORP_GEOG_CTRY_GRP_NM','AC_TYP_GRP','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

#Checked that none are too small
df_groupby_all_ttl[df_groupby_all_ttl.pax_count_ttl > 0].head()


,GMT_UPLIFT_DAY_OF_WEEK,GMT_PLND_DEP_HOUR,CORP_GEOG_CTRY_GRP_NM,AC_TYP_GRP,pax_count_ttl
0,Monday,1. Early Morning,BENELUX,NB,20872
18,Monday,1. Early Morning,EASTERN EUROPE,NB,32440
19,Monday,1. Early Morning,EASTERN EUROPE,WB - First,5629
30,Monday,1. Early Morning,IRELAND RO,NB,6908
36,Monday,1. Early Morning,MEDITERRANEAN,NB,61695


Finalising Lookup Table

In [42]:
list_groupby = ['GMT_UPLIFT_DAY_OF_WEEK','GMT_PLND_DEP_HOUR','CORP_GEOG_CTRY_GRP_NM','AC_TYP_GRP','Lounge_eligibility_tier']

# grouping by GMT_UPLIFT_DAY_OF_WEEK and Lounge_eligibility_tier
df_groupby_final = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()


# columns that you want to group by
list_groupby = ['GMT_UPLIFT_DAY_OF_WEEK','GMT_PLND_DEP_HOUR','CORP_GEOG_CTRY_GRP_NM','AC_TYP_GRP']

# grouping by WB_NB_CAT
df_groupby_final_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_final = pd.merge(df_groupby_final,
                            df_groupby_final_ttl,
                            on = ['GMT_UPLIFT_DAY_OF_WEEK','GMT_PLND_DEP_HOUR','CORP_GEOG_CTRY_GRP_NM','AC_TYP_GRP'],
                            how = 'left'
                           )

df_groupby_final['pax_eligible%'] = (df_groupby_final['pax_count'] / df_groupby_final['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_final['pax_eligible%'] = df_groupby_final['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_final.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

#Drop non eligible
mask = df_groupby_final['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_final = df_groupby_final[~mask].copy()

df_groupby_final.head()
df_groupby_final = df_groupby_final.drop(df_groupby_final[df_groupby_final['pax_eligible%'] == 'nan%'].index) #remove nan%s

# using set_index to come up with the final lookup table
df_groupby_final = df_groupby_final.set_index(['GMT_UPLIFT_DAY_OF_WEEK','GMT_PLND_DEP_HOUR','CORP_GEOG_CTRY_GRP_NM','AC_TYP_GRP','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')


df_groupby_final.head(30)

pax_eligible%  \
Lounge_eligibility_tier                                                               Tier 1   
GMT_UPLIFT_DAY_OF_WEEK GMT_PLND_DEP_HOUR CORP_GEOG_CTRY_GRP_NM   AC_TYP_GRP                    
Monday                 1. Early Morning  BENELUX                 NB                     1.5%   
                                         EASTERN EUROPE          NB                     0.6%   
                                                                 WB - First             4.9%   
                                         IRELAND RO              NB                     1.4%   
                                         MEDITERRANEAN           NB                     0.6%   
                                                                 WB - First             5.7%   
                                         NORTH AFRICA            NB                     0.1%   
                                         SCANDINAVIA             NB                     1.3%   
                                         SOUTH ASIA              WB - First             5.7%   
                                         UK                      NB                     0.6%   
                                         USA                     WB - First             6.4%   
                                         WEST EUROPE             NB                     1.0%   
                       2. Morning        BENELUX                 NB                     0.8%   
                                         CANADA                  WB - First             6.2%   
                                                                 WB - No First          0.5%   
                                         CARIBBEAN               WB - First             0.0%   
                                                                 WB - No First          0.3%   
                                         CHINA                   WB - First             6.2%   
                                         EAST AND CENTRAL AFRICA WB - First             4.0%   
                                         EASTERN EUROPE          NB                     0.7%   
                                         FAR EAST OTHER          WB - First             1.6%   
                                                                 WB - No First          0.8%   
                                         IRELAND RO              NB                     0.6%   
                                         JAPAN                   WB - First             6.2%   
                                         MEDITERRANEAN           NB                     0.5%   
                                         NORTHERN GULF           WB - First             5.3%   
                                         SAUDI ARABIA            WB - First             5.0%   
                                         SCANDINAVIA             NB                     0.7%   
                                         SOUTH AMERICA           WB - First             0.1%   
                                                                 WB - No First          0.2%   

                                                                                       \
Lounge_eligibility_tier                                                        Tier 2   
GMT_UPLIFT_DAY_OF_WEEK GMT_PLND_DEP_HOUR CORP_GEOG_CTRY_GRP_NM   AC_TYP_GRP             
Monday                 1. Early Morning  BENELUX                 NB             14.9%   
                                         EASTERN EUROPE          NB              7.1%   
                                                                 WB - First     13.7%   
                                         IRELAND RO              NB             12.7%   
                                         MEDITERRANEAN           NB              7.3%   
                                                                 WB - First      9.8%   
                                         NORTH AFRICA            NB              6.2%   
                    

# What level of granularity do you use?

We've decided to split all flights by 4 metrics

- **Day of Week**: Weekends have lower proportion of eligible customers (72% compared to 70-71%) with Monday-Wednesday having a higher proportion of customers in Tier 1 compared to other days (2.2% vs 2.0%) due to different profile of travel (e.g. business vs leisure) on each day. 

- **Time of Day**: Time of day will have differing eligibility splits due to profile of customers at each time of day (e.g. business vs leisure). The early morning is more likely to have customers in Tier 2 and less likely to have customers in Tier 1 while the evening has the smallest proportion of customers eligible and lowest proportion in Tier 3.

- **Country Group Name**: Splitting by route would be too low-level as this may change season-on-season, while there is significant difference in behaviour by continent (e.g. Canada has 2.8% in Tier 1 while USA has 5.0%). Therefore we use country group which should be low-level enough to capture differences in behaviour but high-level enough to not be affected by schedule changes, where we can accurately approximate gaps.

- **AC Type Group**: This is a key split as the amount of premium cabin capacity available on a flight will have a significant effect on the eligibility of customers as e.g. First customers contribute to Tier 1. 


What metric do you use to come up with Lounge eligibility profiles?

Summed up all passengers for each of these four splits + tier and divided by total number of passengers in tier